[![gammapy](https://img.shields.io/badge/powered%20by-gammapy-orange.svg?style=flat)](https://gammapy.org/)

# Gammapy Part I: Overview
**Tutors:** Rubens Costa Jr and M Felipe Sousa.

This hands-on tutorial gives an introduction and overview of [Gammapy](https://gammapy.org/).
In the first part of the this tutorial we will learn about the basic data structures in Gammapy using [Third Fermi-LAT Catalog of High-Energy Sources (3FHL) catalog](http://fermi.gsfc.nasa.gov/ssc/data/access/lat/3FHL/), in the second part we will perform an analysis of the Galactic center using simulated CTA data.

## Preface
We recommend to follow this tutorial by **executing the code cells on your local machine**, along with the tutor. The estimated time for this part of the tutorial is ~60 minutes.

We're happy to receive any **feedback or questions** on the tutorial via mail to *rubensjrcosta@gmail.com* or *manoelfelipesousa@gmail.com*.


<a id='intro'></a>
## Indice
* [**1. What is Gammapy?**](#gammapy)

* [**2. Setup**](#setup)

* [**3. Event lists**](#elists)
  * We will learn how to handle event lists with Gammapy. Important for this are the following classes:
    - `~gammapy.data.EventList`
    - [astropy.table.Table](http://docs.astropy.org/en/stable/api/astropy.table.Table.html)
    
* [**4. Sky maps**](#smaps)
  * We will learn how to handle image based data with gammapy using a Fermi-LAT 3FHL example image. We will work with the following classes:
    - `~gammapy.maps.WcsNDMap`
    - [astropy.coordinates.SkyCoord](http://astropy.readthedocs.io/en/latest/coordinates/index.html)
    - [numpy.ndarray](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html)





<a id='gammapy'></a>
## 1. What is Gammapy?


[![Gammapy](https://docs.gammapy.org/0.20.1/_images/gammapy_banner.png)](https://gammapy.org/)


Gammapy is an open-source Python package for gamma-ray astronomy built on [Numpy](http://www.numpy.org/), [Scipy](http://www.scipy.org/) and [Astropy](http://www.astropy.org/). It is base library for the future Science Tools of the [Cherenkov Telescope Array](http://cta-observatory.org/) (CTA). It is already used for the analysis of [High Energy Stereoscopic System](https://www.mpi-hd.mpg.de/hfm/HESS/) (H.E.S.S.), [The Magic Telescopes](https://magic.mpp.mpg.de) (Magic), [Very Energetic Radiation Imaging Telescope Array System](https://veritas.sao.arizona.edu) (VERITAS),
[High-Altitude Water Cherenkov Observatory](https://www.hawc-observatory.org) (HAWC)
and [Fermi Gamma-ray Space Telescope](https://fermi.gsfc.nasa.gov) (Fermi-LAT) data.

The **Gammapy package is structured into sub-packages**, check [documentation](https://docs.gammapy.org/0.18.2).

___

<a id='setup'></a>
🔝 [Back to Top](#intro)<br>
## 2. Setup

In [ ]:
!pip install gammapy

In [ ]:
from gammapy.utils.check import check_tutorials_setup

check_tutorials_setup()

**Important**: to run this tutorial the environment variable `GAMMAPY_DATA` must be defined and point to the directory on your machine where the datasets needed are placed. To check whether your setup is correct you can execute the following cell:

In [ ]:
import os # Miscellaneous operating system interfaces
os.environ['GAMMAPY_DATA'] = os.path.join(os.getcwd(), '/content/gammapy-data/1.2')

In [ ]:
try:
    from gammapy.data import DataStore
    DataStore.from_dir("$GAMMAPY_DATA/hess-dl3-dr1")
    print("Check setup: OK")
except: print("Error")

Tell us about any errors you come across!

Now we can continue with the usual IPython notebooks and Python imports:

In [ ]:
# Display figures directly inline
%matplotlib inline

import matplotlib.pyplot as plt # A collection of command style functions

In [ ]:
# I'd like to ignore some deprecation warnings by matplotlib
# This in general not advised, but for this specific notebook
from matplotlib import MatplotlibDeprecationWarning # A class for issuing deprecation warnings

import warnings # Warning control
warnings.filterwarnings(
    "ignore", category=MatplotlibDeprecationWarning
)

In [ ]:
import astropy.units as u # Defines, converts between, and performs arithmetic with physical quantities (meters, seconds, Hz, etc) and logarithmic units (magnitude and decibel)
from astropy.coordinates import SkyCoord # Representation, manipulation, and transformation between systems of celestial coordinates

___

<a id='elists'></a>
🔝 [Back to Top](#intro)<br>
## 3. Event lists

Almost any high level gamma-ray data analysis starts with the raw measured counts data, which is stored in event lists. In Gammapy event lists are represented by the `~gammapy.data.EventList` class.

In this section we will learn how to:

* Read event lists from FITS files
* Access and work with the `EventList` attributes such as `.table` and `.energy`
* Filter events lists using convenience methods

You will find the documentation of the EventList  class in [gammapy.data.EventList](https://docs.gammapy.org/1.2/api/gammapy.data.EventList.html) or by typing in the command line `help(EventList)`.

Let's start with the import from the `~gammapy.data` submodule:

In [ ]:
from gammapy.data import EventList

In [ ]:
help(EventList)

An event list can be created, by passing a filename to the `~gammapy.data.EventList.read()` method. Let's created one by reading the Fermi-LAT 3FHL event list:

In [ ]:
events_3fhl = EventList.read(
    "$GAMMAPY_DATA/fermi-3fhl-gc/fermi-3fhl-gc-events.fits.gz"
)

In [ ]:
print(events_3fhl)

This time the actual data is stored as an [astropy.table.Table](http://docs.astropy.org/en/stable/api/astropy.table.Table.html) object. It can be accessed with `.table` attribute:

<a id='table1'></a>

In [ ]:
events_3fhl.table

In [ ]:
type(events_3fhl.table)

[Click here](#table1d) to see the descriptions of the reconstructed parameters.

<b>TIP</b><br>Get help on the available readers for `Table` using the``help()`` method:
```python
Table.read.help()  # Get help reading Table and list supported formats
Table.read.help('fits')  # Get detailed help on Table FITS reader
Table.read.list_formats()  # Print list of available formats
```

In [ ]:
# from astropy.table import Table
# Table.read.help('fits')

In [ ]:
# help(Table)

You can do *len* over event_3fhl.table to find the total number of events.

In [ ]:
print(f"The total number of events: {len(events_3fhl.table):.0f}")

In [ ]:
# events_3fhl.peek()

In [ ]:
# events_3fhl.table.info

And we can access any other attribute of the `Table` object as well:

In [ ]:
events_3fhl.table.colnames

In [ ]:
type(events_3fhl.table.colnames)

In [ ]:
events_3fhl.table["ENERGY"]

In [ ]:
energy = events_3fhl.table.columns[0]
energy

For convenience we can access the most important event parameters as properties on the `EventList` objects. The attributes will return corresponding Astropy objects to represent the data, such as [astropy.units.Quantity](http://docs.astropy.org/en/stable/api/astropy.units.Quantity.html), [astropy.coordinates.SkyCoord](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html) or [astropy.time.Time](http://docs.astropy.org/en/stable/api/astropy.time.Time.html#astropy.time.Time) objects:

In [ ]:
events_3fhl.energy

In [ ]:
type(events_3fhl.energy)

In [ ]:
events_3fhl.energy.unit

In [ ]:
events_3fhl.energy.to("GeV")

In [ ]:
events_3fhl.energy.unit

<b>TIP</b><br>To transform the column units in the table:
```python
print(events_3fhl.energy.unit) # Print the ENERGY unit
events_3fhl.table["ENERGY"] = events_3fhl.energy.to("GeV")  # Transform the ENERGY to the GeV units
print(events_3fhl.energy.unit) # Print the new ENERGY unit
```

In [ ]:
# print(events_3fhl.energy.unit)
# events_3fhl.table["ENERGY"] = events_3fhl.energy.to("GeV")
# print(events_3fhl.energy.unit)

In [ ]:
events_3fhl.galactic

In [ ]:
events_3fhl.radec

In [ ]:
events_3fhl.time

There is also some convenience to plot the events:

In [ ]:
events_3fhl.plot_image()

In addition `EventList` provides convenience methods to filter the event lists.

One possible use case is to find the highest energy event within a radius of 0.5 deg around the vela position:

In [ ]:
from gammapy.utils.regions import SphericalCircleSkyRegion

In [ ]:
# defines the center (vela position)
vel_pos_lat = "0d"
vel_pos_lon = "0deg"

frame="galactic"
center = SkyCoord(vel_pos_lat, vel_pos_lon, frame=frame)
print(center)

In [ ]:
# defines the region (center: vela position)
radius = 0.5 * u.deg # defines the radius of the region
region = SphericalCircleSkyRegion(center, radius) # defines the region

In [ ]:
print(region)

In [ ]:
# help(SphericalCircleSkyRegion)

In [ ]:
# select all events within a radius of 0.5 deg around center
events_gc_3fhl = events_3fhl.select_region(region)

In [ ]:
events_gc_3fhl.table

In [ ]:
print(f"Total number of counts in the region: {len(events_gc_3fhl.table):.0f}")

In [ ]:
# sort events by energy
events_gc_3fhl.table.sort("ENERGY")

In [ ]:
events_gc_3fhl.table

In [ ]:
# and show highest energy photon
events_gc_3fhl.energy[-1].to("TeV")

🔝 [Back to Top](#intro)<br>

<a id='smaps'></a>
## 4. Sky Maps

The `~gammapy.maps` package contains classes to work with sky images and cubes.

In this section, we will use a simple 2D sky image and will learn how to:

* Read sky images from FITS files
* Smooth images
* Plot images
* Cutout parts from images

In [ ]:
from gammapy.maps import Map

In [ ]:
# help(Map)

In [ ]:
gc_3fhl = Map.read("$GAMMAPY_DATA/fermi-3fhl-gc/fermi-3fhl-gc-counts.fits.gz")
gc_3fhl

In [ ]:
# help(Map.create)

The image is a `~gammapy.maps.WcsNDMap` object:

In [ ]:
gc_3fhl

The shape of the image is 400 x 200 pixel and it is defined using a cartesian projection in galactic coordinates.

The ``geom`` attribute is a `~gammapy.maps.WcsGeom` object:

In [ ]:
print(gc_3fhl.geom)

Let's take a closer look a the `.data` attribute:

In [ ]:
gc_3fhl.data

That looks familiar! It just an *ordinary* 2 dimensional numpy array,  which means you can apply any known numpy method to it:

In [ ]:
print(f"Total number of counts in the image: {gc_3fhl.data.sum():.0f}")

To show the image on the screen we can use the ``plot`` method.

In [ ]:
gc_3fhl.plot(stretch="log", cmap="inferno"); # stretch="sqrt", stretch="linear" and stretch="log"

The ``plot`` method basically calls [plt.imshow](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html), passing the `gc_3fhl.data` attribute but in addition handles axis with world coordinates using [astropy.visualization.wcsaxes](https://docs.astropy.org/en/stable/visualization/wcsaxes/) and defines some defaults for nicer plots (e.g. the colormap 'afmhot'):

In [ ]:
ax = gc_3fhl.plot(stretch="sqrt");
ax.grid(color='white', ls='solid')
ax.set_xlabel('Galactic Longitude (deg)')
ax.set_ylabel('Galactic Latitude (deg)')
ax.scatter(359.94423568, -0.04616002, transform=ax.get_transform('galactic'), s=450,
           edgecolor='blue', facecolor='none') # Marker at Sag A*

To make the structures in the image more visible, we will smooth the data using a Gaussian kernel.

In [ ]:
gc_3fhl_smoothed = gc_3fhl.smooth(kernel="gauss", width=0.2 * u.deg)

In [ ]:
gc_3fhl_smoothed.plot(stretch="sqrt");

The smoothed plot already looks much nicer, but still the image is rather large. As we are mostly interested in the inner part of the image, we will cut out a quadratic region of the size 9 deg x 9 deg around Vela. Therefore we use `~gammapy.maps.Map.cutout` to make a cutout map:

In [ ]:
# define center and size of the cutout region
center = SkyCoord(0, 0, unit="deg", frame="galactic")
gc_3fhl_cutout = gc_3fhl_smoothed.cutout(center, 9 * u.deg)
gc_3fhl_cutout.plot(stretch="sqrt");

For a more detailed introduction to `~gammapy.maps`, take a look a the [maps.ipynb](../api/maps.ipynb) notebook.

Let's check how the data change with the energy.

In [ ]:
from gammapy.maps import MapAxis

In [ ]:
# help(MapAxis)

In [ ]:
# Creates an axis: 10 GeV-2 TeV with 5 bins in energy
energy_axis = MapAxis.from_energy_bounds(
    energy_min="10 GeV",
    energy_max="2 TeV",
    nbin=5
)

In [ ]:
print(energy_axis)

In [ ]:
# Creates an WcsNDMap:
gc_3fhl_cube = Map.create(
    width=(20 * u.deg, 10 * u.deg),
    skydir=center, # Coordinate of map center
    proj="CAR", # Cartesian projection
    binsz=0.05 *u.deg, # Pixel size in degrees
    map_type="wcs", # {'wcs', 'wcs-sparse', 'hpx', 'hpx-sparse', 'region'}
    frame="galactic", # Galactic ("galactic") or Equatorial ("icrs")
    axes=[energy_axis]
)

In [ ]:
print(gc_3fhl_cube)

In [ ]:
print(gc_3fhl)

In [ ]:
print(f"Total number of counts in the image: {gc_3fhl_cube.data.sum():.0f}")

In [ ]:
# Fill in the events on the map and plot them.
gc_3fhl_cube.fill_events(events_3fhl)

In [ ]:
print(f"Total number of counts in the image: {gc_3fhl_cube.data.sum():.0f}")

To make the structures in the image more visible we will smooth the data using a Gaussian kernel.

In [ ]:
gc_3fhl_cube_smoothed = gc_3fhl_cube.smooth(
    kernel="gauss", width=0.1 * u.deg
)

To visualise the data cube we can use interactive plotting:

In [ ]:
gc_3fhl_cube_smoothed.plot_interactive(cmap="inferno")

Or plot the image in energy bands as a grid:

In [ ]:
gc_3fhl_cube_smoothed.plot_grid(
    ncols=3, figsize=(12, 5), cmap="inferno", stretch="sqrt"
);

Why are there five plots in all?

In [ ]:
energy_axis = MapAxis.from_energy_bounds(
    energy_min="10 GeV",
    energy_max="2 TeV",
    nbin=3
)
gc_3fhl_cube = Map.create(
    width=(20 * u.deg, 10 * u.deg),
    skydir=center,
    proj="CAR",
    binsz=0.05 *u.deg,
    map_type="wcs",
    frame="galactic",
    axes=[energy_axis]
)
gc_3fhl_cube.fill_events(events_3fhl)
gc_3fhl_cube_smoothed = gc_3fhl_cube.smooth(
    kernel="gauss", width=0.1 * u.deg
)
gc_3fhl_cube_smoothed.plot_grid(
    ncols=3, figsize=(12, 5), cmap="inferno", stretch="sqrt"
);

___

<a id='smodels'></a>
🔝 [Back to Top](#intro)<br>

<a id='table1d'></a>

The description of the reconstructed parameters in the event list<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1):

## Table 1

|Index| Event Parameter (units) |Description |
|:-|  :-|:- |
|0|ENERGY (MeV) | Reconstructed energy of the event |
|1|RA (degrees) | Reconstructed direction of the event in Right Ascension |
|2|DEC (degrees) | Reconstructed direction of the event in Declination|
|3|L (degrees) | Reconstructed direction of the event in Galactic Longitude|
|4|B (degrees) | Reconstructed direction of the event in Galactic Latitude|
|5|THETA (degrees) | Reconstructed angle of incidence of the event with respect to the LAT boresight (+Z axis of the spacecraft - the line normal to the top surface of the LAT)|
|6|PHI (degrees) | Reconstructed angle of incidence of the event with respect to the +X axis (the line normal to the sun-facing side of the spacecraft)|
|7|ZENITH_ANGLE (degrees) | Angle between the reconstructed event direction and the zenith line (originates at the center of the Earth and passes through the center of mass of the spacecraft)|
|8|EARTH_AZIMUTH_ANGLE (degrees) | Angle of the reconstructed event direction with respect to North (line from spacecraft origin to north celestial pole) as projected onto a plane normal to the zenith. The angle is measured in degrees east of north, such that 90 degrees indicates that the event originated from the west|
|9|TIME (seconds) | Mission elapsed time when the event was detected (MET is the total number of seconds since 00:00:00 on January 1, 2001 UTC)|
|10|EVENT_ID | Sequence number for the event in the LAT data acquisition period|
|11|RUN_ID | Unique identifier for each LAT data acquisition period|
|12|RECON_VERSION | Version of event reconstruction software in use at the time the event was detected|
|13|CALIB_VERSION (3-element array) | Version of the calibration tables for the ACD, CAL, and TKR (in that order) in use at the time the event was detected. (This column is currently unused)|
|14|EVENT_CLASS | A bitfield indicating which event class selections a given event has passed. In Pass 8 the internal FITS format of this column has been changed from a 32-bit integer (TFORMn=J) to a 32-bit bit column (TFORMn=32X) and supports bitwise selections with the fselect FTOOL. Pass 8 populates a much larger number of bit values than Pass 7. Bits for the recommended event classes are bit 4 (P8R2_TRANSIENT020), bit 7 (P8R2_SOURCE), and bit 10 (P8R2_ULTRACLEANVETO)|
|15|EVENT_TYPE | A bitfield indicating which event type selections a given event has passed. This column is a 32-bit bit column (TFORMn=32X) and supports bitwise selections with fselect.
|16|CONVERSION_TYPE | Indicates whether the event induced pair production in the front (thin) layers or the back (thick) layers of the tracker (front=0, back=1)|
|17|LIVETIME (seconds) | A short-term measure of accumulated livetime of the LAT. This value can have gaps and it resets every few seconds. For large time intervals, the LIVETIME documented in the spacecraft file is correct. However, for short time intervals, this LIVETIME value can be compared between two events to gauge the fraction of dead time|
|18|DIFRSP0 | Diffuse response for an additional component (currently unused)|
|19|DIFRSP1 | Diffuse response for an additional component (currently unused)|
|20|DIFRSP2 | Diffuse response for an additional component (currently unused)|
|21|DIFRSP3 | Diffuse response for an additional component (currently unused)|
|22|DIFRSP4 | Diffuse response for an additional component (currently unused)|



🔝 [Back to Table 1](#table1)<br>

### Other Resources
Gammapy already includes a variety of [tutorial notebooks](https://docs.gammapy.org/0.18.2/tutorials/index.html) covering many analysis scenarios for gamma-ray data including computation of images, spectra and "cubes", spectro-morphological modelling of sources, combined analyses of multiple instruments etc.

🔝 [Back to Top](#intro)<br>

## References
<a name="cite_note-1"></a>1. [](#cite_ref-1) Cicerone: Data — LAT Data Files - Column Descriptions. Retrieved [November 5, 2022] from https://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/Cicerone_Data/LAT_Data_Columns.html.

<a name="cite_note-2"></a>2. [](#cite_ref-2)M. Ajello et al. [Fermi-LAT Collaboration], TFHL:
The third catalog of hard Fermi-LAT sources, Astrophys.
J. Suppl. 232, no. 2, 18 (2017) doi:10.3847/1538-4365/aa8221
[arXiv:1702.00664v3 [astro-ph.HE]].